In [13]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, norm, poisson, gamma
import ipywidgets as widgets
from ipywidgets import interact

fig = None

# Функция для построения графиков распределений
def plot_distribution(distribution, mode, **kwargs):
    global fig
    
    fig = plt.figure(figsize=(10, 6))
    
    sample_size = kwargs.get('sample_size', 1000)
    
    if distribution == 'Равномерное':
        a = kwargs['a']
        b = kwargs['b']
        
        if mode == 'Теоретическое':
            x = np.linspace(a - (b - a) * 0.1, b + (b - a) * 0.1, sample_size)
            pdf = uniform.pdf(x, loc=a, scale=b-a)
            cdf = uniform.cdf(x, loc=a, scale=b-a)
            plt.plot(x, pdf, label='PDF')
            plt.plot(x, cdf, label='CDF')
            plt.title(f'Равномерное распределение (a={a}, b={b})')
        else:
            sample = uniform.rvs(loc=a, scale=b-a, size=sample_size)
            n = int(np.floor(np.log2(sample_size)) + 2)
            plt.hist(sample, bins=n, density=True, alpha=0.6, label='Гистограмма')
            plt.plot(np.sort(sample), np.arange(1, sample_size+1)/sample_size, label='Эмпирическая CDF')
            plt.title(f'Равномерное распределение (a={a}, b={b}) - Случайная выборка')
        
    elif distribution == 'Нормальное':
        mu = kwargs['mu']
        sigma = kwargs['sigma']
        
        if mode == 'Теоретическое':
            x = np.linspace(mu - 4 * sigma, mu + 4 * sigma, sample_size)
            pdf = norm.pdf(x, loc=mu, scale=sigma)
            cdf = norm.cdf(x, loc=mu, scale=sigma)
            plt.plot(x, pdf, label='PDF')
            plt.plot(x, cdf, label='CDF')
            plt.title(f'Нормальное распределение (μ={mu}, σ={sigma})')
        else:
            sample = norm.rvs(loc=mu, scale=sigma, size=sample_size)
            n = int(np.floor(np.log2(sample_size)) + 2)
            plt.hist(sample, bins=n, density=True, alpha=0.6, label='Гистограмма')
            plt.plot(np.sort(sample), np.arange(1, sample_size+1)/sample_size, label='Эмпирическая CDF')
            plt.title(f'Нормальное распределение (μ={mu}, σ={sigma}) - Случайная выборка')
        
    elif distribution == 'Пуассона':
        lam = kwargs['lambda']
        
        if mode == 'Теоретическое':
            x = np.arange(0, lam * 3 + 10)
            pmf = poisson.pmf(x, mu=lam)
            cdf = poisson.cdf(x, mu=lam)
            plt.bar(x, pmf, alpha=0.7, label='PMF')
            plt.step(x, cdf, label='CDF', where='mid', color='tab:orange')
            plt.title(f'Распределение Пуассона (λ={lam})')
        else:
            sample = poisson.rvs(mu=lam, size=sample_size)
            plt.hist(sample, bins=range(0, int(max(sample))+2), density=True, alpha=0.6, label='Гистограмма')
            plt.step(np.sort(sample), np.arange(1, sample_size+1)/sample_size, label='Эмпирическая CDF', where='mid')
            plt.title(f'Распределение Пуассона (λ={lam}) - Случайная выборка')

    elif distribution == 'Эрланговское':
        k = kwargs['k']
        lam = kwargs['lambda']
        
        if mode == 'Теоретическое':
            x = np.linspace(0, gamma.ppf(0.99, k, scale=1/lam), sample_size)
            pdf = gamma.pdf(x, a=k, scale=1/lam)
            cdf = gamma.cdf(x, a=k, scale=1/lam)
            plt.plot(x, pdf, label='PDF')
            plt.plot(x, cdf, label='CDF')
            plt.title(f'Эрланговское распределение (k={k}, λ={lam})')
        else:
            sample = gamma.rvs(a=k, scale=1/lam, size=sample_size)
            n = int(np.floor(np.log2(sample_size)) + 2)
            plt.hist(sample, bins=n, density=True, alpha=0.6, label='Гистограмма')
            plt.plot(np.sort(sample), np.arange(1, sample_size+1)/sample_size, label='Эмпирическая CDF')
            plt.title(f'Эрланговское распределение (k={k}, λ={lam}) - Случайная выборка')

    plt.xlabel('x')
    plt.ylabel('Вероятность')
    plt.legend()
    plt.grid(True)
    plt.show()

# Функция для выбора параметров и построения графика
def select_distribution(distribution):
    mode_selector = widgets.RadioButtons(
        options=['Теоретическое', 'Случайное'],
        value='Теоретическое',
        description='Режим:',
        disabled=False
    )
    
    sample_size = widgets.IntSlider(value=1000, min=100, max=10000, step=100, description='Размер выборки:')
    
    if distribution == 'Равномерное':
        a = widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='a')
        b = widgets.FloatSlider(value=1, min=-10, max=10, step=0.1, description='b')
        ui = widgets.VBox([mode_selector, a, b, sample_size])
        out = widgets.interactive_output(plot_distribution, {'distribution': widgets.fixed(distribution), 'mode': mode_selector, 'a': a, 'b': b, 'sample_size': sample_size})
        display(ui, out)

    elif distribution == 'Нормальное':
        mu = widgets.FloatSlider(value=0, min=-10, max=10, step=0.1, description='μ')
        sigma = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description='σ')
        ui = widgets.VBox([mode_selector, mu, sigma, sample_size])
        out = widgets.interactive_output(plot_distribution, {'distribution': widgets.fixed(distribution), 'mode': mode_selector, 'mu': mu, 'sigma': sigma, 'sample_size': sample_size})
        display(ui, out)

    elif distribution == 'Пуассона':
        lam = widgets.FloatSlider(value=4, min=0.1, max=20, step=0.1, description='λ')
        ui = widgets.VBox([mode_selector, lam, sample_size])
        out = widgets.interactive_output(plot_distribution, {'distribution': widgets.fixed(distribution), 'mode': mode_selector, 'lambda': lam, 'sample_size': sample_size})
        display(ui, out)

    elif distribution == 'Эрланговское':
        k = widgets.IntSlider(value=3, min=1, max=10, step=1, description='k')
        lam = widgets.FloatSlider(value=1, min=0.1, max=10, step=0.1, description='λ')
        ui = widgets.VBox([mode_selector, k, lam, sample_size])
        out = widgets.interactive_output(plot_distribution, {'distribution': widgets.fixed(distribution), 'mode': mode_selector, 'k': k, 'lambda': lam, 'sample_size': sample_size})
        display(ui, out)

# Виджет для выбора распределения
distribution_selector = widgets.Dropdown(
    options=['Равномерное', 'Нормальное', 'Пуассона', 'Эрланговское'],
    value=None,
    description='Распределение:',
    style={'description_width': 'initial'}
)

widgets.interact(select_distribution, distribution=distribution_selector)


interactive(children=(Dropdown(description='Распределение:', options=('Равномерное', 'Нормальное', 'Пуассона',…

<function __main__.select_distribution(distribution)>

In [21]:
fig.savefig('emp_erlang.pdf', bbox_inches='tight', transparent=True)